In [1]:
import sys
sys.path.insert(0, '../' )


In [2]:
import m_fri_ldt as fri

###########  test   #############

proof = fri.ldt_gen_proof( [i+5 for i in range(16)] , b'123' , RS_rho=32 )


rho: 32, offset: 0x8000000000000000
|original poly| : 16

### commit phase ###
?? Assume the evaluate on [0x8000000000000000,|16|x32) has been committed ?
|vi|: 512
derive x0 <- H( h_state || 3 || 1 ): 0x5b9610dcbdcfa2f0b480abae7e2526ddad75cf20f9becf1b
iteration 0: [16] -> [8]:
do a redundent commit[0] here for checking correctness
commit evaluated valuse. --> commits[0] <- |mesg|: 256
|commits| = 1
update h_state <- H( h_state|| xi ): b'\xf8\xff?\x90]\xfe\x94?jM\n\x0cH\xba\x04nU\xd0\xed\xe5De3r<-n\x9fL\xd2R\xf6'
update i: 1
derive new challenge xi <- H(h_state||4||1) : 0xc2699f897fff266d98801881da3468244cf53838795df1ef
deriving new polynomial of length [8]
ibtfy_1( |v0|:512 , 0x8000000000000000) ->
xi:0xc2699f897fff266d98801881da3468244cf53838795df1ef * |vi|:256 ->
evaluated values generated by 1 stage of ibtfy  [256]
iteration 1: [8] -> [4]:
commit evaluated valuse. --> commits[1] <- |mesg|: 128
|commits| = 2
update h_state <- H( h_state|| xi ): b'\x94:T\xbb\x9a\xf1W@\xb6\xee\xda\x17

In [3]:


#def ldt_verify( proof , _poly_len , h_state , Nq = 26 , RS_rho = 8 , verbose = 1 ):

############# test  ##############

fri.ldt_verify( proof , 16 , b'123' , RS_rho = 32 )

######## recovery hash state and challenges ########
derive x0 <- H( h_state || 3 || 1 ): 0x5b9610dcbdcfa2f0b480abae7e2526ddad75cf20f9becf1b
iteration 0: [16] -> [8]:
mt.root = commits[0] =  b'\xa6WO\xaf\xe9n]\x8f\xc9]\x9dg:\x07\xdc\xad\x91_A\x84\xda\xa4g\xae\x1b){\x8d[)$\xf9'
update h_state <- H( h_state|| xi ): b'\xf8\xff?\x90]\xfe\x94?jM\n\x0cH\xba\x04nU\xd0\xed\xe5De3r<-n\x9fL\xd2R\xf6'
update i: 1
derive new challenge xi <- H(h_state||4||1) : 0xc2699f897fff266d98801881da3468244cf53838795df1ef
new polynomial length [8]
iteration 1: [8] -> [4]:
mt.root = commits[1] =  b'b\x0c\xa6\xb6\xe2G@\\n\x91\xcc\x88\xb7% \xda\xd9\xa6\xfan\xedB#\x80hV\x16t\x90\xf0\x00"'
update h_state <- H( h_state|| xi ): b'\x94:T\xbb\x9a\xf1W@\xb6\xee\xda\x17\x13\xa3\x7fW\xee\xde\x89\xb4\xd1m\x9bj)T\x1a\x0b\xa4\x02f\x9d'
update i: 2
derive new challenge xi <- H(h_state||5||1) : 0x60bf68e365f0747c6446e96830a1b8df0b15c2b29ab39427
new polynomial length [4]
iteration 2: [4] -> [2]:
mt.root = commits[2] =  b'*S8\x1

True